## FVA simulations to detect glycine decarboxylase activity 

### Imports

In [6]:
import sys
from cobra.io import read_sbml_model
sys.path.append("../Code")
from model_functions import *
from cobra.flux_analysis import flux_variability_analysis
from cobra import flux_analysis

### Importing and constraining the model

In [7]:
"""
2nd set of constrains

"""

c4_model = read_sbml_model("../Models/c4_model.xml")

c4_model.solver = "glpk"

c4_model.objective = '[M]_Maize_biomass_tx'

#Constrains for light dependent maintenance costs
atp_b = c4_model.reactions.get_by_id("[B]_ATPase_tx")
photon_b = c4_model.reactions.get_by_id("[B]_Photon_tx")
atp_m = c4_model.reactions.get_by_id("[M]_ATPase_tx")
photon_m = c4_model.reactions.get_by_id("[M]_Photon_tx")

const_b = c4_model.problem.Constraint((0.0049 * photon_b.flux_expression + 2.7852) - atp_b.flux_expression , lb = 0, ub = 0)
c4_model.add_cons_vars(const_b)

const_m = c4_model.problem.Constraint((0.0049 * photon_m.flux_expression + 2.7852) - atp_m.flux_expression , lb = 0, ub = 0)
c4_model.add_cons_vars(const_m)


# ATP/NADPH 3:1 constraints
const = c4_model.problem.Constraint(c4_model.reactions.get_by_id("[B]_ATPase_tx").flux_expression -  3 * (c4_model.reactions.get_by_id("[B]_NADPHoxc_tx").flux_expression + c4_model.reactions.get_by_id("[B]_NADPHoxp_tx").flux_expression + c4_model.reactions.get_by_id("[B]_NADPHoxm_tx").flux_expression) , lb = 0, ub = 0)
c4_model.add_cons_vars(const)

const = c4_model.problem.Constraint(c4_model.reactions.get_by_id("[M]_ATPase_tx").flux_expression -  3 * (c4_model.reactions.get_by_id("[M]_NADPHoxc_tx").flux_expression + c4_model.reactions.get_by_id("[M]_NADPHoxp_tx").flux_expression + c4_model.reactions.get_by_id("[M]_NADPHoxm_tx").flux_expression) , lb = 0, ub = 0)
c4_model.add_cons_vars(const)



No objective coefficients in model. Unclear what should be optimized


#### Experiment specific Rubisco constraints

In [8]:

#Force C4 cycle: Block Rubisco carboxylase/oxygenase in Mesophyll and exterior BS Rubisco

set_bounds('[B]_RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p_Ex', (0, 0), c4_model)

set_fixed_flux_ratio({'[M]_RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p':3,'[M]_RXN_961_p':1},c4_model)
set_fixed_flux_ratio({'[B]_RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p':10,'[B]_RXN_961_p':1},c4_model)

### Flux Simulation

#### FVA for GDC with volume constraint

In [9]:
"""
Model simulation (autotrofic) with pFBA

Generation of a solution frame with resulting fluxes
"""

reaction_list = ["[B]_GCVMULTI_RXN_m", "[M]_GCVMULTI_RXN_m"]

with c4_model:
    c4_model.objective = "[M]_Maize_biomass_tx"
    c4_model.solver = "glpk"
    medium = c4_model.medium
    medium["[M]_CO2_tx"] = 40   #CO2 Uptake constrain
    medium["[B]_Nitrate_tx"] = 1000 #Nitrate uptake constrain
    c4_model.medium = medium

    #Volume Constrains
    volume_ratio = 5.3                                                                                                                                                                                   

    #Light Uptake
    light = 1000
    light_ratio = 0.66


    #Equations for defining the light uptake
    set_bounds("[M]_Photon_tx", ((light/(light_ratio+1)), light/(light_ratio+1)), c4_model)
    set_bounds("[B]_Photon_tx", ((light-(light/(light_ratio+1)))/volume_ratio, (light-(light/(light_ratio+1)))/volume_ratio), c4_model)


    #Defining volume constrain in the exchange reactions
    for reaction in c4_model.reactions:
        if reaction.id[0:4] == "[MB]":
            dict = {reaction.products[0] : -(1/volume_ratio)}
            reaction.subtract_metabolites(dict, combine=False)

    
    #Running FVA
    solution = flux_variability_analysis(c4_model, reaction_list=reaction_list, pfba_factor=1)
    print(solution)
    solution = flux_analysis.pfba(c4_model)
    solution_frame=solution.to_frame()


                    minimum       maximum
[B]_GCVMULTI_RXN_m   1.4437  1.443700e+00
[M]_GCVMULTI_RXN_m   0.0000  1.899731e-12


#### FVA for GDC without volume constraint

In [10]:
with c4_model:
    c4_model.objective = "[M]_Maize_biomass_tx"
    c4_model.solver = "glpk"
    medium = c4_model.medium
    medium["[M]_CO2_tx"] = 40   #CO2 Uptake constrain
    medium["[B]_Nitrate_tx"] = 1000 #Nitrate uptake constrain
    c4_model.medium = medium

    #Volume Constrains
    volume_ratio = 1                                                                                                                                                                                

    #Light Uptake
    light = 1000
    light_ratio = 0.66
    
    #Equations for defining the light uptake
    set_bounds("[M]_Photon_tx", ((light/(light_ratio+1)), light/(light_ratio+1)), c4_model)
    set_bounds("[B]_Photon_tx", ((abs(light-(light/(light_ratio+1))))/volume_ratio, (abs(light-(light/(light_ratio+1))))/volume_ratio), c4_model)

    #Defining volume constrain in the exchange reactions
    for reaction in c4_model.reactions:
        if reaction.id[0:4] == "[MB]":
            dict = {reaction.products[0] : -(1/volume_ratio)}
            reaction.subtract_metabolites(dict, combine=False)

    solution = flux_variability_analysis(c4_model, reaction_list=reaction_list, pfba_factor=1)
    print(solution)
    solution = flux_analysis.pfba(c4_model)
    solution_frame=solution.to_frame()

                     minimum   maximum
[B]_GCVMULTI_RXN_m  5.935902  5.935902
[M]_GCVMULTI_RXN_m  1.858851  1.858851
